In [2]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
import setting
import tweepy
import os

twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)


In [3]:
# !pip install tweepy

In [6]:
keyword = "#バンキシャ"
q = keyword
#認証
auth = tweepy.OAuthHandler( setting.CONSUMER_KEY, setting.CONSUMER_SECRET )
auth.set_access_token( setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True)

#検索キーワード設定 
q = keyword

#つぶやきを格納するリスト
tweets_data =[]

#カーソルを使用してデータ取得
for tweet in tweepy.Cursor(api.search, q=q, count=100,tweet_mode='extended').items():

    #つぶやきテキスト(FULL)を取得
    tweets_data.append(tweet.full_text + '\n')

#出力ファイル名
dfile = "../data/hash_tag"
fname = "init.txt"
if not os.path.exists( dfile):
    os.mkdir(dfile)

#ファイル出力
with open(dfile+"/"+fname, "w",encoding="utf-8") as f:
    f.writelines(tweets_data)

# print
for i in range(len(tweets_data)):
    print(tweets_data[i])

最近インスタでshiseのそよ風ってやつめっちゃ流れてくるねんけど
melの彗星と街にそっくりってか丸かぶりやから
この件について詳しく追求していきます
#バンキシャ

RT @togatoga0158: バンキシャ前に夏目アナを祝福する桝太一アナ。
#桝太一 
#日本テレビ 
#アナウンサー 
#ZIP 
#バンキシャ 
#おめでとうございます https://t.co/imvVkgF4Zt

バンキシャ前に夏目アナを祝福する桝太一アナ。
#桝太一 
#日本テレビ 
#アナウンサー 
#ZIP 
#バンキシャ 
#おめでとうございます https://t.co/imvVkgF4Zt

RT @mmmkkkmmm00: 絶対に絶対に違うんだけど、自分の心が腐っているので、桝アナの後ろのセットが中指立てているように見える。

#バンキシャ https://t.co/Z0zoHx61nq

#ATゲート 閉じました。
#番記者 #バンキシャ https://t.co/7gk7qITVZv

RT @mmmkkkmmm00: 絶対に絶対に違うんだけど、自分の心が腐っているので、桝アナの後ろのセットが中指立てているように見える。

#バンキシャ https://t.co/Z0zoHx61nq

RT @mmmkkkmmm00: 絶対に絶対に違うんだけど、自分の心が腐っているので、桝アナの後ろのセットが中指立てているように見える。

#バンキシャ https://t.co/Z0zoHx61nq

RT @yuno_0301_pocco: え、バンキシャリニューアルしたと思ったら中指立ててる…………？？？
#バンキシャ https://t.co/nDu9HswWYq

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

RT @tomoharu_hata: 桝アナの後ろに「中指サイン」が？　日テレ「バンキシャ！」新セットにまさかの憶測 https://t.co/UlkeDbPDAo #バンキシャ！ #リニューアル #中指 #桝太一 #福沢朗 @jcast_newsから

桝アナの後ろに「中指サイン」が？　日テレ「バンキシャ！」新セットにまさかの憶測 https://t.co/UlkeDbPDAo #バン


【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

【為になる！】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @nikkansports: 夏目アナ、有吉の「仲間を大事にする所にひかれた」 
#夏目三久 #有吉弘行 #バンキシャ https://t.co/2UDi7fZ9rh

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

RT @nikkansports: 夏目アナ、有吉の「仲間を大事にする所にひかれた」 
#夏目三久 #有吉弘行 #バンキシャ https://t.co/2UDi7fZ9rh

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

RT @SMARTNET21: バンキシャ（冒頭）
＃バンキシャ https://t.co/O6O80U4pFd

#バンキシャ　夏目ちゃんの幸せそうな笑顔がとても素敵！

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @khot23: #バンキシャ 夏目三久の満面の笑顔よ。 https://t.co/sFtxhia13m

RT @modelpress: 夏目三久、有吉弘行と結婚後初生出演で祝福に感謝

#バンキシャ

▼記事詳細はこちら

https://t.co/svDLRCoNhQ

RT @Master_Bird_: バンキシャより夏目三久アナの幸せコメントほっこりする夏目アナ夏目ちゃんやっぱり可愛い有吉さんと幸せに！！

#バンキシャ

 https://t.co/cf494ZyWbH

RT @TONKO216: 夏目さん本人の結婚報告
#夏目三久 #バンキシャ https://t.co/miBb0YsuKR

RT @nittele_ntv: ＼毎日当たる／
#BangBang新番特番 キャンペーン✨

今夜6時からの『#バンキシャ』の実況や感想を投稿して【プライベート・ビエラ】を当てよう🎁
①@ni


知事がパフォーマンスで披露したマスクの使い方
それより、黙って食べる方が自然

 #バンキシャ

RT @doorknobsmz: 古市「なんかスタジオが朗らかですよね。福澤さんがいなくなったせいか、夏目さんが結婚したせいか」
桝「間違いなく後者だと思います」
#バンキシャ

今までの #バンキシャ 、
小骨程度のホネはあったが、もう完全に骨抜きされたんだね。

いや、自ら骨抜きにしたのかな？

RT @nittele_ntv: ＼毎日当たる／
#BangBang新番特番 キャンペーン✨

今夜6時からの『#バンキシャ』の実況や感想を投稿して【プライベート・ビエラ】を当てよう🎁
①@nittele_ntvをフォロー
②↓のボタンからツイート

フォロー＆RTキャンペーン…

一定の睡眠時間とテレビの時間。
すげぇ分かる。これクリア出来れば、永遠に自粛生活でも可能だな。私は( ＾ω＾ )元々出不精だし。
#バンキシャ

RT @Le_cristal1: 「やってる感」全開の
神奈川県の統括官。
その割には結果が出ていないのはなぜ？🤔
#バンキシャ

RT @doorknobsmz: 古市「なんかスタジオが朗らかですよね。福澤さんがいなくなったせいか、夏目さんが結婚したせいか」
桝「間違いなく後者だと思います」
#バンキシャ

ワクチンが十分供給できるようにすることを考えるのが一番大事と言った古市氏、ズレてますね。日本ですぐ大量生産できますか？感染しないよう＆させないようにすることは必要です。#バンキシャ

神奈川でギスギスしてるから大阪はもっと忙しいのかな？
 #バンキシャ

神奈川のコロナ対策統括官に密着する桝太一までは良いだろう。更にそれを写すカメラは要らん。自画自賛か？良い企画だって誉めて欲しいのか？番組はどこを見ている？#バンキシャ

RT @mn_enta_tv: 考えてみれば、3月まで桝さんと夏目さんって裏同士だったんですよね。

#バンキシャ

実績の無い神奈川のプロパガンダしてどうする？
先行き不安だね、桝くん。
チャンネル変えるね。
#バンキシャ

素人のおっさんの1日見せられて誰得なのこれ #バンキシャ

散々東京をフューチャーして煽っておいて、なぜいきなり神奈川なのか？
小池んとこ行けよwww
#バンキシャ

バンキシャ!が全てを持っていく…


数年ぶりにオープニングから観たわ

 #バンキシャ

ほうちゅうさんがイジられて若干照れてるのが可愛い

#バンキシャ #大塚芳忠

伊沢さんと桝さんが10年ぶりの…！
#バンキシャ

さすが古市くん！
キャラ出している？
#バンキシャ

#バンキシャ
夏目さん幸せそうだよ🌟
良かったね🌟 https://t.co/rOfPmkDRKL

ニュースの合間か…。
しゃーない。
お風呂入ってこよ…。

録画はしたんだ。夫にも録画せよと言われたんだ。でもその前に録画したんだ。夫婦で好きすぎなんだよ。。どっちも推しなの。。
#バンキシャ

#バンキシャ 
『ニュースの合間の良き所』(笑) 
８８８回の記念すべき回にお目出度い報道だね～。心こもってない(笑)

夏目さんの幸せな表情が全て
#バンキシャ!

芳忠さん｢夏目さん！私からもおめでとうございます！｣
夏目さん｢芳忠さんのような素敵な夫婦になれるように頑張っていきます｣
#バンキシャ

古市ワロタ
#バンキシャ

@poe1985 夏目さんご結婚💒ほんとうにおめでたいです💕

古市さんと枡さんとのやり取りも楽しみ♬
ニヤニヤ😊

#バンキシャ

バンキシャ（冒頭）
＃バンキシャ https://t.co/O6O80U4pFd

芳忠さん、人柄も素敵〜 #バンキシャ

伊沢くんきた〜

#バンキシャ #伊沢拓司

ジャストミートが居なくなったおかげで現場がピリついてない　#バンキシャ

RT @doorknobsmz: 夏目さんおめでとうございます！ #バンキシャ

もういいから早くニュースやって
#バンキシャ

RT @delchil: 天の声といってるのに大塚芳忠さんの名を #バンキシャ

なんて爽やかなの！
#バンキシャ　#桝太一

芳忠さん、相変わらず英国紳士のようなお声。
#バンキシャ

古市いらねー。
#バンキシャ

わぁい🙌、桝さんだ〜💕
#バンキシャ

ニュースの間の良きところで…

#バンキシャ

RT @mml7BiNOwTnz0rk: #バンキシャ #夏目三久 #桝太一
おめでとうございます( ^-^)ノ∠※。.:*:・'°☆ https://t.co/n7TQX4gwHW

饅頭先輩のお嫁さんめっちゃ綺麗やんけ！！！#バンキシャ

【番組について一言】 #バンキシャ #BangBang新番特番 @